<a href="https://colab.research.google.com/github/philipjpark/Product-Recommendation/blob/main/ProductRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
!pip install kafka-python
!apt-get install -y librdkafka-dev
!pip install confluent-kafka
from kafka import KafkaConsumer, KafkaProducer

kz = pd.read_csv("/content/kz.csv").drop_duplicates()
#on_bad_lines skips certain lines

kz['event_time'] = pd.to_datetime(kz['event_time']).dt.date

kz.shape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
librdkafka-dev is already the newest version (1.2.1-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


(363170, 8)

[Using Kafka Producer to Consumer Messaging](https://www.youtube.com/watch?v=DDYQ1eXhk3E)

In [ ]:
kz.columns.values.tolist()

['event_time',
 'order_id',
 'product_id',
 'category_id',
 'category_code',
 'brand',
 'price',
 'user_id']

In [ ]:
data_types= kz.dtypes

print(data_types)


event_time        object
order_id           int64
product_id         int64
category_id      float64
category_code     object
brand             object
price            float64
user_id          float64
dtype: object


Here I am adding up all the instances in which a user made a purchase. Then I am arranging it from the most voluminous purchasers to the least and printing out the data types of each feature. 

In [ ]:
kz.groupby('user_id', as_index=False).size().sort_values(by="size", ascending=False)

,user_id,size
21859,1.515916e+18,209
21802,1.515916e+18,198
21848,1.515916e+18,193
21801,1.515916e+18,192
21845,1.515916e+18,189
...,...,...
9441,1.515916e+18,1
9445,1.515916e+18,1
9448,1.515916e+18,1
9449,1.515916e+18,1


In [ ]:
train = kz.loc[kz['event_time'] < pd.Timestamp('2020-09-01'),:]

valid = kz.loc[kz['event_time'] >= pd.Timestamp('2020-09-01'),:]

<ipython-input-12-2a62dca231f9>:1: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  train = kz.loc[kz['event_time'] < pd.Timestamp('2020-09-01'),:]
<ipython-input-12-2a62dca231f9>:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  valid = kz.loc[kz['event_time'] >= pd.Timestamp('2020-09-01'),:]


In [ ]:
import numpy as np

def avg_prec(actual, predicted, k=15):
    if len(predicted)>k:
        predicted = predicted[:k]  #limiting the number of predicted items to the top-k items. 

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)   #here we have a check for the predicted items to see if they are in the actual items list

    if not actual:
        return 0.0

    return score / min(len(actual), k)
def mean_avg_prec(actual, predicted, k=10):
    return np.mean([avg_prec(a,p,k) for a,p in zip(actual, predicted)])
    
actual_items = [3, 7, 4, 2, 5]
actual_items = [str(i) for i in actual_items]
recommended_items = [12, 7, 53, 90, 3, 23, 14, 37, 18, 67]
recommended_items = [str(i) for i in recommended_items]

avg_prec(actual_items, recommended_items) 
#The average precision of a recommendation system based on the actual items that the user interacted with ('actual_items') 
#Then we have items that the recommendation system recommended to the user ('recommended_items').


0.18

**Approach Based on Naive (General)**

In [ ]:
train

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18
5,2020-04-26,2295716521449619559,1515966223509261697,2.268105e+18,furniture.kitchen.table,maestro,39.33,1.515916e+18
6,2020-04-26,2295740594749702229,1515966223509104892,2.268105e+18,electronics.smartphone,apple,1387.01,1.515916e+18
...,...,...,...,...,...,...,...,...
363325,2020-02-05,2348786187084234824,2273948311037673564,2.268105e+18,NaN,polimerbiht,1.83,NaN
363326,2020-02-05,2348786187084234824,1515966223527009208,NaN,0.81,NaN,NaN,NaN
363327,2020-02-05,2348786187084234824,1515966223512083926,2.268105e+18,NaN,NaN,3.91,NaN
363328,2020-02-05,2348786187084234824,1515966223510394021,2.268105e+18,furniture.living_room.shelving,none,10.39,NaN


In [ ]:
valid_df = valid.groupby('user_id', as_index=False).agg({'product_id': list})
valid_df

,user_id,product_id


In [ ]:
train_df = train.groupby('user_id', as_index=False).agg({'product_id': list}, ascending=False)
train_df

,user_id,product_id
0,1.515916e+18,[1515966223509089059]
1,1.515916e+18,[1515966223509089642]
2,1.515916e+18,[1515966223509088567]
3,1.515916e+18,"[2273948227772351313, 1515966223509088620, 151..."
4,1.515916e+18,[1515966223509107036]
...,...,...
22296,1.515916e+18,[1515966223509089982]
22297,1.515916e+18,[1515966223510207823]
22298,1.515916e+18,"[1515966223509132221, 1515966223511247107]"
22299,1.515916e+18,[2273948190023614610]


In [ ]:
popular = train.drop_duplicates(['product_id', 'user_id']).groupby('product_id', as_index=False).size().sort_values('size', ascending=False)
popular
#Drops any duplicates in which the we will only have 1 row per user-product interaction

,product_id,size
61,1515966223509088620,291
128,1515966223509089045,259
19,1515966223509088521,218
289,1515966223509089486,214
12673,2273948316473492113,204
...,...,...
7456,1515966223510397178,1
7455,1515966223510397174,1
7454,1515966223510397152,1
7452,1515966223510397133,1


In [ ]:
valid_df = valid_df.merge(train_df, on='user_id')
valid_df

,product_id_x,user_id,product_id_y


In [ ]:
valid_df['pred'] = [[i for i in list(popular['product_id'][:15].values) if i not in j][:3] for j in valid_df['product_id_y']]
valid_df
#This is now outputting recommendations based on a list of the top 15 popular items then filters out the top 3 items that the user has not already interacted with

,product_id_x,user_id,product_id_y,pred


**Approach Based on Rating**

In [ ]:
product_ratings = train.groupby('product_id', as_index=False).size()['size'] 
#number of product rating reviews

num_ratings = train.drop_duplicates(['product_id', 'user_id']).groupby('product_id', as_index=False).size()['size'] 
#number of unique users that reviewed products but also removing duplicates

last_rating = train.groupby('product_id', as_index=False).agg({'event_time': max})
#most recent review made for the product. 

In [ ]:
rating_count_df = pd.DataFrame({'number of ratings': product_ratings, 'unique user reviews': num_ratings})
rating_count_df = rating_count_df.join(last_rating).sort_values(by="unique user reviews", ascending=False)

rating_count_df

,number of ratings,unique user reviews,product_id,event_time
61,329,291,1515966223509088620,2020-07-08
128,829,259,1515966223509089045,2020-07-08
19,935,218,1515966223509088521,2020-07-08
289,842,214,1515966223509089486,2020-07-08
12673,2238,204,2273948316473492113,2020-07-08
...,...,...,...,...
7456,2,1,1515966223510397178,2020-02-04
7455,1,1,1515966223510397174,2020-07-02
7454,1,1,1515966223510397152,2020-01-24
7452,1,1,1515966223510397133,2020-01-13


In [ ]:
def create_ranked_df(train):
        product_ratings = train.groupby('product_id', as_index=False).size()['size']
        num_ratings = train.drop_duplicates(['product_id', 'user_id']).groupby('product_id', as_index=False).size()['size']
        last_rating = train.groupby('product_id', as_index=False).agg({'event_time': max}).rename(columns={'event_time': 'last_rating'})
        
        rating_count_df = pd.DataFrame({'avg_rating': product_ratings, 'num_ratings': num_ratings})
        rating_count_df = rating_count_df.join(last_rating)

        recs =  train.loc[:, ['product_id', 'category_id', 'category_code', 'brand', 'price']].drop_duplicates().merge(rating_count_df, on=['product_id'])

        ranked = recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

        ranked = ranked[ranked['num_ratings'] > 4] 
        #filters out products with less than 5 reviews because it is not popular enough to be considered
        
        return ranked
    

def popular_recommendations(user_id, n_top, ranked):

    top = list(ranked['product_id'][:n_top])

    return top
#The function generates popular recommendations for a given user based on the previously established metrics.

In [ ]:
ranked = create_ranked_df(train)

popular_recommendations('1', 20, ranked)

[1515966223509117074,
 2273948186349404174,
 1515966223523303307,
 2273948316473492113,
 1515966223509106786,
 2273948218662322995,
 2273948216221238252,
 2273948218662322996,
 1515966223509089067,
 1515966223509089598,
 2273948222722409184,
 1515966223509088521,
 2273948227654910595,
 1515966223509089283,
 2273948227654910594,
 1515966223509131886,
 1515966223523303304,
 1515966223509089486,
 1515966223509089646,
 1515966223509089045]

***Modified Version of Project Leader Leon's Code***
  

In [ ]:
import threading
import time
from queue import Queue

class Message:
#Update this class with the content: userid, productid, event_time
    def __init__(self, userid: float, productid: int, event_time: object):
        self.userid = userid
        self.productid = productid
        self.event_time = event_time
    def __str__(self):
        return f"Message:\n\tUserID: {self.userid}\n\tProductID {self.productid}\n\tEvent_Time: {self.event_time}"


class KafkaBroker:
    def __init__(self, topic):
        self.topic = topic
        self.data = {}

    def produce_message(self, message: Message):
  
        if self.topic in self.data:
            self.data[self.topic].append(message)
        else:
            self.data[self.topic] = [message]

    def consume_messages(self):
        if self.topic in self.data and len(self.data[self.topic]) > 0:
            message = self.data[self.topic].pop(0)
            return message
        return None

        time.sleep(.001)

class KafkaProducer:
    def __init__(self, broker, topic, user_id, product_id, event_time, ranked):
        self.broker = broker
        self.topic = topic
        self.user_id = user_id
        self.product_id = product_id
        self.event_time = event_time
        self.ranked = ranked

    def send(self, message: Message):
        kafka_message = Message(message.userid, message.productid, message.event_time)
        self.broker.produce_message(kafka_message)

class KafkaConsumer:
    def __init__(self, broker, topic, user_id, n_top, ranked):
        self.broker = broker
        self.topic = topic
        self.queue = Queue()
        self.thread = threading.Thread(target=self.run)
        self.thread.start()
        self.counter = 0
        self.user_id = user_id
        self.n_top = n_top
        self.ranked = ranked

    def run(self):
        while True:
            message = self.broker.consume_messages()
            if message:
                self.queue.put(message)
                self.counter += 1
                print(f"Consumed message --{self.counter}")
                if self.counter % 1000 == 0:
                    print("Calculate recommendation")
                    self.calculate_and_output_recommendations(self.user_id)

    def poll(self, timeout=1):
        try:
            return self.queue.get(timeout=timeout)
        except:
            return None

    def calculate_and_output_recommendations(self, user_id, n_top):
        top_n = popular_recommendations(user_id, n_top, self.ranked)
        precision = ranked(user_id, top_n)
        print(f"Top-{n_top} recommendations for user {user_id}: {top_n}")
        print(f"Precision for user {user_id}: {precision}")

        
    def recommendation_output(self, message: Message):
        print(f"UserID of Recommended Item: {message.userid}")
        print(f"ProductID of Recommended Item: {message.productid}")
        print(f"Event_Time of Recommended Item: {message.event_time}")

        
if __name__ == "__main__":
    broker = KafkaBroker('test_topic')
    producer = KafkaProducer(broker, 'test_topic', 'user_id', 'product_id','event_time', ranked)
    consumer = KafkaConsumer(broker, 'test_topic', 'user_id', 3, ranked)

    message = Message('user_id', 'product_id', ranked)
    producer.send(message)

    timeoutSeconds = 5
    start_time = time.time()
    while (time.time() - start_time) < timeoutSeconds:
        time.sleep(0.2)
        msg = consumer.poll()
        if msg is not None:
            consumer.recommendation_output(msg)

  

Consumed message --1
UserID of Recommended Item: user_id
ProductID of Recommended Item: product_id
Event_Time of Recommended Item:                product_id   category_id                category_code  \
1151  1515966223509117074  2.268105e+18                          NaN   
6937  2273948186349404174  2.268105e+18                          NaN   
7205  1515966223523303307  2.374499e+18       electronics.smartphone   
361   2273948316473492113  2.268105e+18       electronics.smartphone   
827   1515966223509106786  2.268105e+18       electronics.smartphone   
...                   ...           ...                          ...   
519   1515966223509127502  2.268105e+18  electronics.audio.headphone   
1991  1515966223509106872  2.268105e+18       electronics.smartphone   
2032  1515966223509553805  2.268105e+18       electronics.smartphone   
2730  1515966223510403240  2.268105e+18       electronics.smartphone   
14    1515966223509105105  2.268105e+18                          NaN   

    

**Explanation of the code:**
Message class: A message that can be sent through the broker. 
It has three attributes: email, type (event), and content (a dictionary representing the data transmitted).


KafkaBroker class: The broker initializes with a topic and an empty dictionary. This enables the storage of messages. that manages the messages. 
- produce_message() method takes a Message object and appends a message to the dictionary of newly added data. 
- consume_messages() method that receives messages from the data dictionary if they exist.


KafkaProducer class has a broker and a topic. 
-send() method that produces a message to the broker by using the produce_method() found in the broker. 


KafkaConsumer class has a broker, topic, user_id, n_top, and ranked function.

-run() method creates a queue to store messages and continuously runs the consume_messages() found in the broker. These messages are placed in the line.This is controlled by threads. It keeps a counter that is incremented with each message. Once the counter reaches 1000, the calculate_and_output_recommendations() is called to calculate and output the recommendations. Once finished, the counter is reset to 0, and the recommendation_output method is called to output the top recommendations and precision for the user.

- poll() method gets the message from the line. The recommendation_output() then calculates and outputs the top_n recommendations based on the precision of "ranked." The ranked variable which stores the create_ranked_df(train) function is supposed to run a precision mechanism to output highly recommended items. I need to work on this to understand what is really going. 

The main block has instances of KafkaBroker, KafkaProducer, and KafkaConsumer. The topic is 'test_topic'. A message is then produced to the broker using a producer instance. The consumer is set to poll for messages for 5 seconds. Messages received will be outputted. 

After 1000 consumed messges, the consumer will calculate and display the top 5 recommendations based on precision from the recommendation_output method. 

